In [12]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [13]:
#import các thư viện cần thiết
import pandas as pd
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
nltk.download('stopwords')
# nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diept\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\diept\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\diept\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

# Preprocess funtion

In [15]:
def merge_files() -> None:
#     """Merge all JSON files of a folder

#     Sample folder structure
#     ```
#     └── 📁data
#         └── 📁merged
#             └── test.json
#         └── 📁processed
#         └── 📁raw
#             └── test01.json
#             └── test02.json
#     ```
#     Args:
#         None
#     Return:
#         returns None"""
    files = os.listdir(os.path.join("..", "data", "raw"))
    text_todf = []
    for i in range(len(files)):
        with open(os.path.join("..", "data", "raw", files[i]), 'r') as file:
            data = json.load(file)
            df_metadata = pd.json_normalize(data, max_level=1)
            text = ""
            for element in df_metadata['body_text'][0]:
                text += element['text']
            text_todf.append(text)
    df = pd.DataFrame({"text": text_todf})
    df.to_json(os.path.join("..", "data", "merged", "corpus.json"))


In [16]:
 #Read data
data = pd.read_json(os.path.join("..", "merged", "test.json"))
data.head()


,text
0,As a consequence of the global COVID-19 pandem...
1,According to current live statistics at the ti...


In [17]:
def preprocess(data : pd.DataFrame) -> pd.DataFrame : 
  """ Performing data preprocessing
  Agrs : 
    data : input file as pandas.DataFrame
  Returns :
    A pandas.DataFrame which is as vector form of input"""

  #Create a copy file so that we wont change the original data
  cleaned_data = data.copy()
  

  stop_words = set(stopwords.words('english'))
  lemmatizer = WordNetLemmatizer()
  def clean_text(text):
     #text cleaning, normalization
    text = text.lower()
    text = re.sub(r'[^a-z\s]', ' ',text)
    text = re.sub(r'\s+',' ',text).strip()
    #Tokenization
    tokens = word_tokenize(text)
    #Text lemmatization, stopword removal
    tokens = [lemmatizer.lemmatize(t) for t in tokens if t not in stop_words]
    return ' '.join(tokens)
  cleaned_data['text_data'] = cleaned_data['text'].apply(clean_text)
  

  #Rare words, common words filtering
  vectorizer = TfidfVectorizer(max_df= 0.9, min_df= 0.2)
  #vecorization using TF-IDF
  tfidf_data = vectorizer.fit_transform(cleaned_data['text_data'])
  #Convert to Dataframe
  cleaned_data = pd.DataFrame(tfidf_data.toarray(),columns = vectorizer.get_feature_names_out())
  return cleaned_data



In [18]:
processed_data = preprocess(data)
processed_data.head()

,ability,able,academy,acceleration,access,accordance,according,account,accuracy,accurate,...,widely,window,work,working,world,worldwide,would,writing,wtft,wtfts
0,0.057213,0.028606,0.028606,0.028606,0.057213,0.028606,0.00000,0.00000,0.114426,0.028606,...,0.028606,0.028606,0.00000,0.00000,0.00000,0.000000,0.114426,0.00000,0.228852,0.028606
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.04131,0.04131,0.000000,0.000000,...,0.000000,0.000000,0.04131,0.04131,0.04131,0.082619,0.000000,0.04131,0.000000,0.000000


In [19]:
#Output csv
processed_data.to_csv(os.path.join('..','cleaned_data.csv'),index =False)

# Training LDA

In [20]:
!pip install pyspark

In [23]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import LDA
from pyspark.sql.functions import col, udf 
from pyspark.sql.types import ArrayType, StringType
from pyspark.ml.feature import VectorAssembler

In [ ]:
#Initialize Pyspark Session
spark = SparkSession.builder \
    .appName("LDA Topic Modeling") \
    .master("local[*]")\
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

In [31]:
#Transform pd.DataFrame to pyspark.sql.DataFrame
spark_data = spark.createDataFrame(processed_data) 

#Combine columns into 1 vector column
assembler = VectorAssembler(inputCols= spark_data.columns, outputCol= 'features')
processed_data_vector = assembler.transform(spark_data).select('features')


In [ ]:
# Initialize number of topics and max interation
num_topics = 10
max_interation = 100

In [ ]:
#Train LDA
lda = LDA(k=num_topics, maxIter=max_interation, featuresCol= 'features')
lda_model = lda.fit(processed_data_vector)
# Show topics 
topics = lda_model.describeTopics(maxTermsPerTopic= 10)
topics.show(truncate=False)

+-----+--------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|topic|termIndices                                       |termWeights                                                                                                                                                                                                                          |
+-----+--------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|0    |[257, 53, 531, 94, 185, 393, 526, 166, 316, 232]  |[0.002556330017035642, 0.002344122630907044, 0.002338477109205982, 0.002331